## Importing things 

In [1]:
import sys
sys.path.append('../misc')

from MoviaBusDataset import MoviaBusDataset
from BaseNetwork import BaseNetwork
import numpy as np
from FNN import FNN
from Seq2Seq import Seq2Seq

%load_ext autoreload
%autoreload 2


## Load Data

In [2]:
previous_timesteps = 6
prediction_steps = 6
batch_size = 25

train = MoviaBusDataset('../data/train', interpolation=True, 
                        prev_timesteps=previous_timesteps, 
                        max_future_time_steps=prediction_steps, 
                        timeofday = True)
train2 = MoviaBusDataset('../data/train', interpolation=True, 
                        prev_timesteps=previous_timesteps, 
                        max_future_time_steps=prediction_steps, 
                        )

validation = MoviaBusDataset('../data/validation', interpolation=True, 
                       prev_timesteps=previous_timesteps, 
                       max_future_time_steps=prediction_steps, 
                       timeofday = True)
validation2 = MoviaBusDataset('../data/validation', interpolation=True, 
                       prev_timesteps=previous_timesteps, 
                       max_future_time_steps=prediction_steps)


test = MoviaBusDataset('../data/test', interpolation=True, 
                       prev_timesteps=previous_timesteps, 
                       max_future_time_steps=prediction_steps, 
                       timeofday = True)
test2 = MoviaBusDataset('../data/test', interpolation=True, 
                       prev_timesteps=previous_timesteps, 
                       max_future_time_steps=prediction_steps)

train.remove_trend()
validation.remove_trend(train._historical_average)
test.remove_trend(train._historical_average)

train2.remove_trend()
validation2.remove_trend(train2._historical_average)
test2.remove_trend(train2._historical_average)

train2.normalize(individual_roads=False)
validation2.normalize(train2.mean, train2.std)
test2.normalize(train2.mean, train2.std)

## FNN

In [3]:
net_FNN = FNN(num_hidden=20)
net_FNN.load('FNN.pt')

validation_scores=[net_FNN.get_MAE_score(validation,timestep=i) for i in range(1,prediction_steps+1)]
print('Validation score:\n',validation_scores)
print('\nMean validation score for all timesteps:\n',sum(validation_scores)/len(validation_scores))

test_scores=[net_FNN.get_MAE_score(test,timestep=i) for i in range(1,prediction_steps+1)]
print('\nTest score:\n',test_scores)
print('\nMean test score for all timesteps:\n',sum(test_scores)/len(test_scores))

RuntimeError: Error(s) in loading state_dict for FNN:
	size mismatch for FNN.0.weight: copying a param of torch.Size([20, 7]) from checkpoint, where the shape is torch.Size([20, 2]) in current model.

In [ ]:
net_FNN.visualize_road(test, timesteps=1, road=17)

## Seq2Seq

In [ ]:
net_s2s = Seq2Seq()
net_s2s._BaseNetwork__target_to_net = True
net_s2s.load('Sequence2Sequence.pt')

validation_scores=[net_s2s.get_MAE_score(validation2,timestep=i) for i in range(1,prediction_steps+1)]
print('Validation score:\n',validation_scores)
print('\nMean validation score for all timesteps:\n',sum(validation_scores)/len(validation_scores))

test_scores=[net_s2s.get_MAE_score(test2,timestep=i) for i in range(1,prediction_steps+1)]
print('\nTest score:\n',test_scores)
print('\nMean test score for all timesteps:\n',sum(test_scores)/len(test_scores))

In [ ]:
time,out_s2s,tar = net_s2s.visualize_road(test2, timesteps=6, road=16)
_,out_fnn,_ = net_FNN.visualize_road(test, timesteps=6, road=16)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

plt.plot(time,out_s2s, label='Truth_s2s')
plt.plot(time,tar, label='Prediction')
plt.plot(time,out_fnn, label='Truth_fnn')
plt.legend()
plt.xlabel('Time of day [MM:DD:HH]')
plt.ylabel('Mean speed [m/2]')
plt.show()

## DCRNN 

In [ ]:
import numpy as np
out_dcrnn = None
with np.load('../dcrnn.npz') as data:
    # also contains groundtruth
    out_dcrnn = np.abs(data['predictions'] - data['groundtruth'])

### Stats

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
stats = pd.read_csv('../stats.csv',index_col=['LinkRef'])
stats['dcrnn_t1'],stats['dcrnn_t2'],stats['dcrnn_t3'],stats['dcrnn_t4'],stats['dcrnn_t5'],stats['dcrnn_t6'] = out_dcrnn.mean(axis=1)
data_norm = (stats.sub(stats.mean(axis=0,numeric_only=True),axis=1))/stats.std(axis=0)
#plt.matshow()
#data_norm
data_norm.corr()


## Plotting